## Health Insurance Cost Prediction

In this data science project, the goal is to predict health insurance charges for patients based on various factors such as age,  BMI, number of children, sex, smoking status, and region. The dataset provides valuable insights into the factors influencing healthcare costs, enabling us to build a predictive model that can assist in estimating charges for new patients.

We can begin by importing the necessary libraries for this project. I will also load in our data and convert it to a pandas dataframe.

In [1]:
import pandas as pd
from scipy.stats import zscore
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
csv_file_path = 'C:\\Users\\nicho\\OneDrive\\Documents\\Projects\\portfolio\\Health Insurance Cost Prediction\\insurance.csv'
charges_df = pd.read_csv(csv_file_path)

An important aspect of any data science project is preprocessing your data. Let's first ensure there are no missing values.

In [2]:
print(charges_df.isnull().sum())

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


This shows we have no missing values and can proceed with checking how our data is formatted.

In [3]:
column_types = charges_df.dtypes
print(column_types)

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object


The data appears to be formatted properly, however we often run into issues with singular values or desire different formats. Let's confirm our columns are consistent.

In [4]:
charges_df['age'] = charges_df['age'].astype(int)
charges_df['sex'] = charges_df['sex'].astype(object)
charges_df['bmi'] = charges_df['bmi'].astype(float)
charges_df['children'] = charges_df['children'].astype(int)
charges_df['smoker'] = charges_df['smoker'].astype(object)
charges_df['region'] = charges_df['region'].astype(object)
charges_df['charges'] = charges_df['charges'].astype(float)

Next, I want to check for any duplicate values.

In [5]:
print('Number of observations: ' + str(len(charges_df)))
charges_df.drop_duplicates(inplace=True)
print('Number of observations: ' + str(len(charges_df)))

Number of observations: 1338
Number of observations: 1337


It appears we did have one duplicate value! This value has been dropped so as to not alter our data exploration. I also want to remove some outliers so there is a more representative mean. This code removes any outliers that are more than 4 standard deviations from the mean for the numerical values in the dataset.

In [6]:
print('Number of observations: ' + str(len(charges_df)))
z_scores = zscore(charges_df[['age', 'bmi', 'children', 'charges']])
charges_df = charges_df[(z_scores < 4).all(axis=1)]
print('Number of observations: ' + str(len(charges_df)))

Number of observations: 1337
Number of observations: 1335


As we can see, we removed 2 values that were classified as outliers. Let's find out what rows were deleted.

In [7]:
orig_charges_df = pd.read_csv(csv_file_path)
merged_df = orig_charges_df.merge(charges_df, how='outer', indicator=True).loc[lambda x: x['_merge'] == 'left_only']
print(merged_df)

      age     sex    bmi  children smoker     region      charges     _merge
544    54  female  47.41         0    yes  southeast  63770.42801  left_only
1300   45    male  30.36         0    yes  southeast  62592.87309  left_only


This table demonstrates the outliers being those with very high medical charges (no age/bmi outliers). The last modification I want to make to the data is to add some categorical variables to better group the observations, as well as add an age group column.

In [8]:
charges_df = pd.get_dummies(charges_df, columns=['sex', 'smoker', 'region'], drop_first=False)
charges_df['age_group'] = pd.cut(charges_df['age'], bins=[0, 26, 42, 58, 100], labels=['Generation Z', 'Millenial', 'Generation X', 'Baby Boomer'])
print('Our new columns are: ' + ', '.join(charges_df.columns))

Our new columns are: age, bmi, children, charges, sex_female, sex_male, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest, age_group


I'm now going to drop a few redundant columns, such as sex_male considering we have sex_female and it is binary.

In [9]:
charges_df = charges_df.drop(columns=['sex_male', 'smoker_no'])

Now I will run some basic SQL queries just to get a sense of the data. We can start by setting up our SQL database.

In [10]:
conn = sqlite3.connect(':memory:')
charges_df.to_sql('charges_table', conn, index=False, if_exists='replace')

1335

What are our top 5 highest charges?

In [11]:
query = '''
        SELECT charges 
        FROM charges_table 
        ORDER BY charges DESC 
        LIMIT 5'''
result_df = pd.read_sql_query(query, conn)
print(result_df)

       charges
0  60021.39897
1  58571.07448
2  55135.40209
3  52590.82939
4  51194.55914


Does age group seem to impact charges?

In [12]:
query = '''
        SELECT age_group, AVG(charges) AS avg_charges
        FROM charges_table 
        GROUP BY age_group 
        ORDER BY avg_charges DESC'''
result_df = pd.read_sql_query(query, conn)
print(result_df)

      age_group   avg_charges
0   Baby Boomer  20824.972901
1  Generation X  15741.348066
2     Millenial  11460.191605
3  Generation Z   8861.064212


What about smoker status?

In [13]:
query = '''
        SELECT smoker_yes AS smoker, AVG(charges) AS avg_charges 
        FROM charges_table 
        GROUP BY smoker'''
result_df = pd.read_sql_query(query, conn)
print(result_df)

   smoker   avg_charges
0       0   8440.660307
1       1  31821.324341


We have our region data. Is there an equal representation from every region in the US?

In [14]:
query = '''
        SELECT
        SUM(CASE WHEN region_southwest = 1 THEN 1 ELSE 0 END) AS count_southwest,
        SUM(CASE WHEN region_northwest = 1 THEN 1 ELSE 0 END) AS count_northwest,
        SUM(CASE WHEN region_southeast = 1 THEN 1 ELSE 0 END) AS count_southeast,
        SUM(CASE WHEN region_northeast = 1 THEN 1 ELSE 0 END) AS count_northeast
        FROM charges_table'''
result_df = pd.read_sql_query(query, conn)
print(result_df)

   count_southwest  count_northwest  count_southeast  count_northeast
0              325              324              362              324


Some takeaways from our queries: 
- Our highest charges are in the 50-60k range
- Charges go up significantly with both age group and smoker status.
- It appears we have roughly the same number of observations from every region. Additionally, each region is represented enough to draw conclusions from.
Let's now take a look at correlations to confirm our suspicions about how some of our features may be impacting charges.

In [15]:
print(charges_df.corr()['charges'].sort_values(ascending=False))

charges             1.000000
smoker_yes          0.787422
age                 0.297719
bmi                 0.192984
children            0.073998
region_southeast    0.064434
region_northeast    0.009563
region_northwest   -0.035667
region_southwest   -0.040668
sex_female         -0.058976
Name: charges, dtype: float64


C:\Users\nicho\AppData\Local\Temp\ipykernel_18244\1118106024.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(charges_df.corr()['charges'].sort_values(ascending=False))


As we can see, smoker status, age, and BMI all have positive correlations with our charges variables, albeit to varying magnitudes. For a final check before we begin to build any models, let's make sure we have enough observations for these binary variables ie. smoker_yes and sex_female.

In [16]:
count_smokers = charges_df['smoker_yes'].sum()
count_non_smokers = len(charges_df) - count_smokers
print('Number of smokers: ' + str(count_smokers) + ', Number of non-smokers: ' + str(count_non_smokers))
count_females = charges_df['sex_female'].sum()
count_males = len(charges_df) - count_females
print('Number of females: ' + str(count_females) + ', Number of males: ' + str(count_males))

Number of smokers: 272, Number of non-smokers: 1063
Number of females: 661, Number of males: 674


Our male to female ratio is exceptional, however I do think we have an underrepresentation of smokers (20% of the data). Let's address this problem before building out our model, so as to not introduce any bias to our highest correlation variable. First, let's split our data into training and testing data for our regression model. We can address the bias later. We need to drop age_group now, as it isn't numerical and therefore cannot be run in a regression. If we wanted to, we could repeat the same process as regions using bins, however this is a bit redundant with our age variable already being included in the data.

In [17]:
X = charges_df.drop(['age_group', 'charges'], axis=1)
y = charges_df['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)
train_charges_df = pd.concat([X_train, y_train], axis=1)
count_smokers = train_charges_df['smoker_yes'].sum()
count_non_smokers = len(train_charges_df) - count_smokers
print('Number of smokers: ' + str(count_smokers) + ', Number of non-smokers: ' + str(count_non_smokers))

Number of smokers: 216, Number of non-smokers: 852


I have decided to perform undersampling here, as bringing the number of non-smokers to 216 keeps us well above the 30 observations per feature guideline. We have 9 features so we need at a minimum 270 observations, and we would have 432 in total. Another methodology for addressing our smoker to non-smoker ratio is oversampling, which could be accomplished using SMOTE, however this is a bit beyond the scope of this project.

In [18]:
nonsmokers_train_df = train_charges_df[train_charges_df['smoker_yes'] == 0]
random_sample = nonsmokers_train_df.sample(n=216, random_state=6)
unbiased_train_charges_df = pd.concat([train_charges_df[train_charges_df['smoker_yes'] == 1], random_sample], axis=0)
new_X_train = unbiased_train_charges_df.drop('charges', axis=1)
new_y_train = unbiased_train_charges_df['charges']

Finally, let's run our model and determine how our features interact with charges.

In [19]:
model = LinearRegression()
model.fit(new_X_train, new_y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
n = X_test.shape[0]
p = X_test.shape[1]
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)
rmse = mean_squared_error(y_test, y_pred, squared=False)
coefficients = model.coef_
intercept = model.intercept_
print('Our r-squared is: ' + str(round(r2, 3)) + ' and our adjusted r-squared is: ' + str(round(adj_r2, 3)))
print('Our root-mean-square error is: ' + str(round(rmse, 2)))
variable_coef = list(sorted(zip(X_train.columns, coefficients), key=lambda x: x[1], reverse=True))
print('Our variables and their associated coefficients:')
for variable, coefficient in variable_coef:
    print(f"{variable}: {round(coefficient, 2)}")
print('Our intercept is: ' + str(round(intercept, 2)))
print(f'Our intercept is: {round(intercept, 2)}')

Our r-squared is: 0.701 and our adjusted r-squared is: 0.69
Our root-mean-square error is: 6534.66
Our variables and their associated coefficients:
smoker_yes: 24070.41
region_northeast: 1074.55
bmi: 746.13
sex_female: 351.25
children: 266.69
age: 265.84
region_southwest: -72.56
region_northwest: -190.46
region_southeast: -811.54
Our intercept is: -25516.54
Our intercept is: -25516.54


Our model demonstrates a moderate level of predictive power, as indicated by an R-squared of 0.701 and an adjusted R-squared of 0.69. The root-mean-square error (RMSE) stands at 6534.66, reflecting the average magnitude of the residuals. Examining the variable coefficients, being a smoker has a substantial positive impact on predicted charges, contributing 24070.41 to the estimate. Other influential variables include region (specifically northeast) and BMI. The intercept, representing the estimated charges when all predictors are zero, is -25516.54.